<a href="https://colab.research.google.com/gist/phineas-pta/ba6cab65a993b43452860a5b94b645cb/gg-colab-ssh-dummy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. preliminary

this notebook shows how to

 - ssh to Google Colab (using [`colab-ssh`](https://github.com/WassimBenzarti/colab-ssh))
 - open new instances of Jupyter (Notebook or Lab), Julia Pluto
 - some settings that may be useful

## 0.0. System info

don't need `sudo` because

In [ ]:
!whoami

the current dir (then we change the default starting dir in shell)

In [ ]:
!pwd

how many threads available (for Julia)

In [ ]:
!lscpu | grep -E '^Thread|^Core|^Socket|^CPU\('

GPU info (if GPU runtime)

In [ ]:
!nvidia-smi -q --display=MEMORY

OS info

In [ ]:
!cat /etc/os-release

## 0.1. Update + install some stuff

hide output with `%%capture` magic

In [ ]:
%%capture
%%bash

apt-get update
apt-get dist-upgrade

apt-get install vim zsh lshw xvfb python-opengl x11-utils # libv8-dev # for rstan

apt-get clean
apt-get autoremove --purge

pip3 install colab_ssh arviz radian watermark prince jill jupyterlab gym pyvirtualdisplay scikit-video nodejs

~40min long when installing Julia, CmdStan & R pkgs

In [ ]:
%%capture
%%bash

pip3 install -U cmdstanpy # replace old version
install_cmdstan -d /opt/cmdstan

R -e 'install.packages(c("FactoMineR", "reticulate", "caret", "keras")) #, "shiny", "plotly"))'
echo "Sys.setenv(RETICULATE_PYTHON='$(which python3)')" | tee -a "$(find / -mount -name Rprofile)"

jill install --confirm # install Julia

# customize Julia (file not existed so create new)
mkdir -p ~/.julia/config
tee ~/.julia/config/startup.jl <<EOT
ENV["PYTHON"] = "$(which python3)"
ENV["CMDSTAN_HOME"] = ENV["JULIA_CMDSTAN_HOME"] = "$(echo /opt/cmdstan/*)" # cmdstan dir is 1 level below
ENV["DATADEPS_ALWAYS_ACCEPT"] = true
EOT

# below cmd: must use double quotes in Julia; bash may collapse new line so
# any comment must be nested and statement must be ended with semicolon
julia --threads 2 -e 'import Pkg; Pkg.add([
    "IJulia", "JSONTables", "StatsBase", "Plots", "DataFrames", "MLBase", "Symbolics", "MLJ",
    "JuMP", "Flux", "StatsPlots", "Distances", "Clustering", "BSON", "MultivariateStats",
    "Turing", "StatsModels", "Stan", "Optim", "TimeSeries", "MLDatasets", "CSV", "XLSX", "GLM",
    "Pluto", "PlutoUI", "OhMyREPL", "Pipe", "PackageCompiler", "PyPlot", "CategoricalArrays",
    "Distributions", "KernelDensity", "DataFramesMeta", "HypothesisTests", "Zygote", "ArviZ",
    "Calculus", "OnlineStats", "Roots", "LaTeXStrings", "PGFPlots", "MCMCChains", "CUDA",
]); Pkg.precompile();
using PackageCompiler, OhMyREPL;
create_sysimage(:OhMyREPL; replace_default = true);'

some misc settings

In [ ]:
%%capture
%%bash

chsh -s $(which zsh) # change shell
sh -c "$(wget -O- https://raw.githubusercontent.com/ohmyzsh/ohmyzsh/master/tools/install.sh)" "" --unattended
git clone https://github.com/zsh-users/zsh-syntax-highlighting.git ~/.oh-my-zsh/plugins/zsh-syntax-highlighting
git clone https://github.com/vim-airline/vim-airline.git ~/.vim/pack/dist/start/vim-airline

# customize zsh with oh-my-zsh
sed -i 's/plugins=(git)/plugins=(colored-man-pages zsh-syntax-highlighting colorize)/;
s/ZSH_THEME="robbyrussell"/ZSH_THEME="agnoster"/;' ~/.zshrc

tee -a ~/.zshrc <<EOT
# some env var
export TF_XLA_FLAGS='--tf_xla_auto_jit=2 --tf_xla_cpu_global_jit' # enable XLA
export JULIA_NUM_THREADS=2 # enable multi-threading in Julia
export CMDSTAN="$(echo /opt/cmdstan/*)" # cmdstan dir is 1 level below

alias ipy3='python3 -m IPython'
alias jupy3='jupyter console --existing /content/colab.json' # save current colab kernel info

# programs to be used with SSH tunneling (change port if you want)
alias pluto='julia -e "using Pluto; Pluto.run(run_notebook_on_load = false, port = 1234, launch_browser = false);"'
alias jupylab='jupyter lab --no-browser --allow-root --port 1234'

# change to current dir
cd /content
EOT

# customize vim (file not existed so create new)
tee ~/.vimrc <<EOT
set visualbell
set noerrorbells
set number
set encoding=UTF-8
set tabstop=4
set shiftwidth=4
let g:airline_powerline_fonts=1
EOT

## 0.2. save current colab kernel info

In [ ]:
import re, json

capture output in `kern` for later use

In [ ]:
%%capture kern
%connect_info

save to JSON file

In [ ]:
colab_kern = json.loads(re.search(
    pattern = r"\{[^}]+}",
    string = kern.stdout
).group())
with open('colab.json', 'w') as outfile:
    json.dump(colab_kern, outfile)

# 1. connect

capture output for later use

In [ ]:
from colab_ssh import launch_ssh_cloudflared

In [ ]:
%%capture connect_ssh
launch_ssh_cloudflared(password = "testoo")

In [ ]:
link = re.search(
    pattern = r"(?<=<code>)[a-z\-]+\.trycloudflare\.com(?=</code>)",
    string = connect_ssh.outputs[0].data['text/html']
).group()

download Cloudflare [here](https://github.com/cloudflare/cloudflared/releases)

create `colab-ssh-config` file with following text:
```
Host *.trycloudflare.com
	HostName %h
	User root
	Port 22
	ProxyCommand <PATH TO>/cloudflared-amd64/cloudflared.exe access ssh --hostname %h
```

In [ ]:
ssh_file = r"D:\w\colab-ssh-config"

## 1.1. SSH

in Windows: remember to enable `OpenSSH`

In [ ]:
print("ssh -F", ssh_file, link)

if you want to use PuTTY: instructions [here](https://developers.cloudflare.com/cloudflare-one/applications/non-HTTP/ssh/putty-clients)

## 1.2. SSH Tunneling

to run Jupyter, Pluto

all use the same port 1234 so do not run all at once (or considering change port for each)

In [ ]:
print("ssh -F", ssh_file, link, "-L 1234:localhost:1234")

free up port before

In [ ]:
!fuser -k 1234/tcp

now you can run `pluto` or `jupylab` in your terminal

don't run here in this colab because `Ctrl+C` in your terminal is better

# 2. keep Colab alive

In [ ]:
1 + 2 # occasionally run

automated solution with JS: open browser inspector view

In [ ]:
%%javascript
function ClickConnect() {
    console.log("Working");
    document.querySelector("colab-connect-button").shadowRoot.getElementById("connect").click();
}
setInterval(ClickConnect, 300000)

for multi-line code block, use:

*   `ctrl+o` **OR**
*   `%cpaste` then paste code



# 3. get plots (or files)

in Julia, use `PyPlot`

## 3.0. change fig size

default `matplotlib figsize = 6.4÷4.8` (4÷3)

In [ ]:
newSize = (8, 6) # (12, 9) # (16, 9)

In [ ]:
import matplotlib as mpl, matplotlib.pyplot as plt, seaborn as sns

In [ ]:
mpl.rc("figure", **{"figsize": newSize, "autolayout": True})

In [ ]:
plt.figure(figsize = newSize, tight_layout = True)

In [ ]:
g = sns.displot(data, bins = "sqrt", kde = True)
g.fig.set_figwidth(newSize[0])
g.fig.set_figheight(newSize[1])

## 3.1 sftp

In [ ]:
print("sftp -F", ssh_file, link)

`savefig` must be executed together with plotting code, not separately

In [ ]:
plt.plot([1,2,3],[4,5,6])
plt.savefig('/content/abc.png')

In [ ]:
!get /content/abc.png Desktop/abc.png

## 3.3 gg drive

2 options

*   use sidebar for GUI
*   CLI below


In [ ]:
from google.colab import drive
from os import environ as osenv

In [ ]:
wdir = '/content/drive/My Drive/Colab Notebooks/'
print(osenv['CLOUDSDK_CONFIG'])

In [ ]:
drive.mount('/content/drive')

In [ ]:
with open(f'{wdir}foo.txt', 'w') as f:
    f.write('Hello Google Drive!')

In [ ]:
plt.plot([1,2,3],[4,5,6])
plt.savefig(f'{wdir}abc.png')

In [ ]:
drive.flush_and_unmount()

# 4. Save object

In [ ]:
import pickle

pickle.dump(link, file = open(f"{wdir}bar.pkl", "wb"), protocol = pickle.HIGHEST_PROTOCOL) # save
link_reloaded = pickle.load(open(f"{wdir}bar.pkl", "rb")) # reload

`pickle` often fails, `dill` can help

In [ ]:
import dill

dill.dump(link, file = open(f"{wdir}bar.pkl", "wb"), protocol = pickle.HIGHEST_PROTOCOL) # save
link_reloaded = dill.load(open(f"{wdir}bar.pkl", "rb")) # reload

# 5. Miscellaneaous

## 5.1. TensorBoard

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/DB-VAE

## 5.2. run R code

In [ ]:
%load_ext rpy2.ipython